In [2]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from model.assumptions_core import read_assumptions
from input.weather.weather import load_weather
from model.network_core  import build_network
from model.constraints_core import calculate_biogas_max

logging.basicConfig(level=logging.INFO)

In [3]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    "EUR": 10.5080,
    "USD": 9.2521
}
base_year = 2024
discount_rate = 0.04
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3
biogas_method = "average"

## Parameters that will change frequently
geo = '14'
self_sufficiency = 0.92
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 0.5
growth_only = True


In [5]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_root / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the demand from csv file
projected_demand_path = paths.demand / f"projected-demand,geography={geo.replace(':','-')},target-year={target_year},growth-only={growth_only}.csv.gz"
demand = pd.read_csv(projected_demand_path, compression='gzip', index_col='timestamp')
target_load = demand['value'].values.flatten()

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.renewables / f"capacity-factor-solar,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.renewables / f"capacity-factor-onwind,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.renewables / f"capacity-factor-offwind,geography={geo},start=2023-01,end=2023-12.nc").values.flatten()

In [7]:
print(capacity_factor_solar.mean())
print(capacity_factor_onwind.mean())
print(capacity_factor_offwind.mean())

0.12895793404707503
0.38274087962810976
0.4369870275835815


In [4]:
# Build the network

biogas = calculate_biogas_max(biogas_limit, target_load, assumptions.loc['combined_cycle_gas_turbine','efficiency'].value, "average")

network = build_network(index, resolution, geography, target_load, assumptions, discount_rate, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

total_e = target_load.sum()

## Add offwind constraint
if False:
    generator_capacity = model.variables["Generator-p_nom"]
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add self-sufficiency constraint on the import market
market_e = model.variables['Generator-p'].loc[:,'market'].sum()
non_sufficiency_e = total_e * (1 - self_sufficiency)
model.add_constraints(market_e <= non_sufficiency_e, name="Self_sufficiency_constraint")

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 779.90it/s]
INFO:linopy.io: Writing time: 0.52s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms


INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 102222 duals
Objective: 1.04e+10
Solver model: available
Solver message: optimal



Coefficient ranges:
  Matrix [4e-06, 1e+02]
  Cost   [6e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e-01, 4e+05]
Presolving model
48185 rows, 45272 cols, 132872 nonzeros  0s
38760 rows, 35847 cols, 120918 nonzeros  0s
38038 rows, 35125 cols, 123282 nonzeros  0s

Solving MIP model with:
   38038 rows
   35125 cols (0 binary, 5 integer, 0 implied int., 35120 continuous)
   123282 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            10384155233.05     Large        0      0      0         0    26.0s
 R       0       0         0   0.00%   10377806335.35  10381960560.78     0.04%        0      0      0     55370    2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance, Self_sufficiency_constraint were not assigned to the network.


('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator biogas        5.000000e-01                 0.0  5.138740e-12   
          import        2.536663e+03                 0.0  1.125000e+06   
          onwind        5.298634e+03                 0.0  7.533134e+06   
          solar         6.529675e+03                 0.0  6.558259e+06   
Link      AC            4.333065e+03                 0.0  1.398387e+07   
          h2            3.961214e-13                 0.0  5.942327e-12   
          li-ion        1.738721e+03                 0.0  2.416764e+06   
          mixedgas      0.000000e+00                 0.0  4.099379e-12   
Load      -             0.000000e+00                 0.0  0.000000e+00   
Store     h2            0.000000e+00                 0.0  5.644983e-12   
          li-ion        1.693096e+04                 0.0  2.510245e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator biogas    4.626896e-12  5.118437e-13  0.000000e+00     2.229597e-15   
          import    0.000000e+00  1.125000e+06  0.000000e+00     5.062740e-02   
          onwind    6.821210e-13  7.533134e+06  0.000000e+00     1.622959e-01   
          solar     0.000000e+00  6.558259e+06  0.000000e+00     1.146550e-01   
Link      AC        1.409139e+07 -1.075223e+05  1.145824e+07     3.712399e-01   
          h2        5.942327e-12  0.000000e+00  0.000000e+00     1.712475e-03   
          li-ion    2.519651e+06 -1.028874e+05  0.000000e+00     1.654270e-01   
          mixedgas  6.595518e-12 -2.496140e-12  5.859482e-12              NaN   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        2.973440e-13  5.347639e-12  0.000000e+00              NaN   
          li-ion    2.516228e+06 -5.983343e+03  0.000000e+00     5.043162e-01   

                     Curtailment  Capital Expenditure  \
Generator biogas    0.000000e+00         0.000000e+00   
          import    0.000000e+00         0.000000e+00   
          onwind    1.023218e+07         4.823795e+09   
          solar     8.181286e+05         2.286391e+09   
Link      AC        0.000000e+00         2.387607e+08   
          h2        0.000000e+00         0.000000e+00   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         0.000000e+00   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         0.000000e+00   
          li-ion    0.000000e+00         2.194516e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator biogas               2.958150e-10      0.0           NaN  
          import               6.750000e+08      0.0           NaN  
          onwind               1.604649e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas             2.791033e-10      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -1.242370e+05      0.0           NaN